In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np


### Loading the data from the directories and preprocess it for training


In [20]:
train_dir = '/Users/olle/Downloads/final_symbols_split_ttv/train'
val_dir = '/Users/olle/Downloads/final_symbols_split_ttv/val'
test_dir = '/Users/olle/Downloads/final_symbols_split_ttv/test'



In [5]:
# Parameters for loading the dataset
batch_size = 32
img_height = 28
img_width = 28

In [22]:
# Load training data
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels='inferred',  # Automatically infer labels from the subdirectory names
    label_mode='int',   # Label as integers (suitable for categorical cross-entropy)
    color_mode='grayscale',
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=True #The dataset should be shuffled randomly each time it is iterated over during training
)

Found 42749 files belonging to 14 classes.


In [23]:
# Load validation data
val_dataset = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    labels='inferred',
    label_mode='int',
    color_mode='grayscale',
    batch_size=batch_size,
    image_size=(img_height, img_width)
)

Found 5337 files belonging to 14 classes.


In [25]:
# Load test data
test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='int',
    color_mode='grayscale',
    batch_size=batch_size,
    image_size=(img_height, img_width)
)

Found 5356 files belonging to 14 classes.


### Preprocess and Prepare for Training